In [1]:
import sys
import os
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
from risk_models.config.read_config.read_func import Read_Oracle
from risk_models.config.write_config.write_func import Write_Oracle
import numpy as np
import datetime
import pandas as pd

In [2]:
ORG_CODE='91310000132612172J'

In [44]:
sql='''select * from BD_RISK_DETAIL_STOCK_ST3 where iscurrent =1 and ORG_CODE = '{}' '''.format(ORG_CODE)
BD_RISK_DETAIL_STOCK_ST3  = Read_Oracle().read_oracle(sql=sql,database='dbods')

2021-07-13 11:15:30.080 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 1.507s


In [48]:
BD_RISK_DETAIL_STOCK_ST3['BUSINESS_DATE'] = BD_RISK_DETAIL_STOCK_ST3['ACTRUAL_STOCK_DATE'].apply(lambda x : '{}年{}月'.format(x.year,x.month))
df = BD_RISK_DETAIL_STOCK_ST3.copy()
df['YEAR'] = df.ACTRUAL_STOCK_DATE.dt.year
measure_list=[]
for key,i in df.groupby('UNIT_CO')['QTY_CO'].sum().sort_values(ascending=False).head(30).items():
    measure_list.append(key)
df = df[df['UNIT_CO'].apply(lambda x : x in measure_list)]
# # 领用
df_take = df[(df['BILL_TYPE'] == 'A') | (df['BILL_TYPE'] == 'D')]
df_back = df[(df['BILL_TYPE'] == 'B') | (df['BILL_TYPE'] == 'C')]
df_back = df_back[['UNIT_CO','QTY_CO','BUSINESS_DATE']].groupby(['BUSINESS_DATE','UNIT_CO'])['QTY_CO'].sum().reset_index()
df_take = df_take[['UNIT_CO','QTY_CO','BUSINESS_DATE']].groupby(['BUSINESS_DATE','UNIT_CO'])['QTY_CO'].sum().reset_index()
df_measure = df_take.merge(df_back,on=['UNIT_CO','BUSINESS_DATE']).rename(columns={'QTY_CO_x':'in_num','QTY_CO_y':'out_num'})
df_measure['VOLUMN'] = df_measure['in_num']+df_measure['out_num']
df_bench = df_measure.groupby(['UNIT_CO'])['VOLUMN'].quantile([0.10,0.90])
df_bench = pd.DataFrame(df_bench).unstack()['VOLUMN']
df_bench= df_bench.reset_index().rename(columns={0.10:'low',0.90:'high'})
df_m1 = df_measure.merge(df_bench,on='UNIT_CO')
df_m1['RISK_LABEL'] = (np.where(df_m1['VOLUMN']<df_m1['low'],'吞吐量偏低',
                        np.where(df_m1['VOLUMN']>df_m1['high'],'吞吐量偏高',
                                                                 '吞吐量正常')))
df_m1 = df_m1.drop(['in_num','out_num','low','high'],axis=1)
# df_m1 = df_m1.drop(['in_num','out_num','low','high','UNIT_CO'],axis=1)

In [49]:
df_m1.sort_values(['BUSINESS_DATE'])

,BUSINESS_DATE,UNIT_CO,VOLUMN,RISK_LABEL
0,2020年10月,M,2638.284,吞吐量偏高
3,2020年10月,M2,65.294,吞吐量正常
6,2020年10月,MM,26719.986,吞吐量偏高
8,2020年10月,码,181.558,吞吐量正常
9,2020年11月,PC,2.815,吞吐量偏低
4,2020年12月,M2,43.420,吞吐量偏低
1,2020年6月,M,537.571,吞吐量正常
11,2020年6月,DM2,2760.518,吞吐量偏低
14,2020年7月,EA,5854.000,吞吐量偏高
12,2020年8月,DM2,11130.572,吞吐量偏高


In [50]:
df = BD_RISK_DETAIL_STOCK_ST3.copy()
# df['TRADE_TOTAL'] = np.random.randint(1000000,size=len(df))

# # 领用
df_take = df[(df['BILL_TYPE'] == 'A') | (df['BILL_TYPE'] == 'D')]
df_back = df[(df['BILL_TYPE'] == 'B') | (df['BILL_TYPE'] == 'C')]
# df_back = df_back[['TRADE_TOTAL','BUSINESS_DATE']].groupby(['BUSINESS_DATE'])['TRADE_TOTAL'].sum().reset_index()
# df_take = df_take[['TRADE_TOTAL','BUSINESS_DATE']].groupby(['BUSINESS_DATE'])['TRADE_TOTAL'].sum().reset_index()
df_back = df_back[['TRADE_TOTAL','BUSINESS_DATE','UNIT_CO']].groupby(['BUSINESS_DATE','UNIT_CO'])['TRADE_TOTAL'].sum().reset_index()
df_take = df_take[['TRADE_TOTAL','BUSINESS_DATE','UNIT_CO']].groupby(['BUSINESS_DATE','UNIT_CO'])['TRADE_TOTAL'].sum().reset_index()
df_m2 = df_take.merge(df_back,on=['BUSINESS_DATE','UNIT_CO']).rename(columns={'TRADE_TOTAL_x':'in_money','TRADE_TOTAL_y':'out_money'})
df_m2['total_money'] = df_m2['in_money'] + df_m2['out_money']
df_m2['TRADE_RATE'] = df_m2['out_money'] / df_m2['total_money'].mean() * 100
df_m2['RISK_LABEL'] = (np.where(df_m2
                           ['TRADE_RATE'] > 50 ,'库存周转率异常','库存周转率正常'))
df_m2 = df_m2.drop(['in_money','out_money','total_money'],axis=1)

In [51]:
df_m2

,BUSINESS_DATE,UNIT_CO,TRADE_RATE,RISK_LABEL
0,2020年10月,M,NaN,库存周转率正常
1,2020年10月,M2,NaN,库存周转率正常
2,2020年10月,MM,NaN,库存周转率正常
3,2020年10月,码,NaN,库存周转率正常
4,2020年11月,PC,NaN,库存周转率正常
5,2020年12月,M2,NaN,库存周转率正常
6,2020年6月,DM2,NaN,库存周转率正常
7,2020年6月,M,NaN,库存周转率正常
8,2020年7月,EA,NaN,库存周转率正常
9,2020年8月,DM2,NaN,库存周转率正常


In [52]:
df_final = df_m1.merge(df_m2,on=['BUSINESS_DATE','UNIT_CO'],how='left')
df_final['ORG_CODE'] = ORG_CODE
now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
df_final['CHECK_TIME']=now
df_final['CHECK_TIME'] = pd.to_datetime(df_final['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')

In [53]:
df_final['SCORE'] =  df_final['RISK_LABEL_x'].apply(lambda x : 0 if '正常' in x else -0.3) + df_final['RISK_LABEL_y'].apply(lambda x : 0 if '正常' in x else -0.3)

In [54]:
df_final['RISK_LABEL'] = df_final['RISK_LABEL_x']+','+df_final['RISK_LABEL_y']

In [55]:
df_final = df_final.drop(['RISK_LABEL_x','RISK_LABEL_y'],axis=1).reset_index().rename(columns={'index':'ID'})
df_final

,ID,BUSINESS_DATE,UNIT_CO,VOLUMN,TRADE_RATE,ORG_CODE,CHECK_TIME,SCORE,RISK_LABEL
0,0,2020年10月,M,2638.284,NaN,91310000132612172J,2021-07-13 11:26:56,-0.3,"吞吐量偏高,库存周转率正常"
1,1,2020年6月,M,537.571,NaN,91310000132612172J,2021-07-13 11:26:56,0.0,"吞吐量正常,库存周转率正常"
2,2,2021年3月,M,60.666,NaN,91310000132612172J,2021-07-13 11:26:56,-0.3,"吞吐量偏低,库存周转率正常"
3,3,2020年10月,M2,65.294,NaN,91310000132612172J,2021-07-13 11:26:56,0.0,"吞吐量正常,库存周转率正常"
4,4,2020年12月,M2,43.420,NaN,91310000132612172J,2021-07-13 11:26:56,-0.3,"吞吐量偏低,库存周转率正常"
5,5,2021年3月,M2,224.479,NaN,91310000132612172J,2021-07-13 11:26:56,-0.3,"吞吐量偏高,库存周转率正常"
6,6,2020年10月,MM,26719.986,NaN,91310000132612172J,2021-07-13 11:26:56,-0.3,"吞吐量偏高,库存周转率正常"
7,7,2021年3月,MM,20532.474,NaN,91310000132612172J,2021-07-13 11:26:56,-0.3,"吞吐量偏低,库存周转率正常"
8,8,2020年10月,码,181.558,NaN,91310000132612172J,2021-07-13 11:26:56,0.0,"吞吐量正常,库存周转率正常"
9,9,2020年11月,PC,2.815,NaN,91310000132612172J,2021-07-13 11:26:56,-0.3,"吞吐量偏低,库存周转率正常"


# 预警表

In [8]:
sql='''select * from BD_RISK_RESULT_STOCK_ST3 where iscurrent =1 and ORG_CODE = '{}' '''.format(ORG_CODE)
df_final  = Read_Oracle().read_oracle(sql=sql,database='dbods')
df_final.head()

2021-06-07 09:27:35.481 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.105s


,ID,ORG_CODE,BUSINESS_DATE,VOLUMN,TRADE_RATE,CHECK_TIME,RISK_LABEL,SCORE,ISCURRENT,LASTUPDATE
0,81,91310000132612172J,2020年10月,2638.284,3.20073,2021-06-04 16:25:41,"吞吐量偏高,库存周转率正常",-0.3,1,2021-06-04 16:25:41
1,82,91310000132612172J,2020年6月,537.571,0.42136,2021-06-04 16:25:41,"吞吐量正常,库存周转率正常",0.0,1,2021-06-04 16:25:41
2,83,91310000132612172J,2021年3月,60.666,62.95414,2021-06-04 16:25:41,"吞吐量偏低,库存周转率异常",-0.6,1,2021-06-04 16:25:41
3,84,91310000132612172J,2020年10月,65.294,3.20073,2021-06-04 16:25:41,"吞吐量正常,库存周转率正常",0.0,1,2021-06-04 16:25:41
4,85,91310000132612172J,2020年12月,43.420,0.85886,2021-06-04 16:25:41,"吞吐量偏低,库存周转率正常",-0.3,1,2021-06-04 16:25:41


In [9]:
ttl_num = df_final['RISK_LABEL'].apply(lambda x: 1 if '正常' not in x.split(',')[0] else 0).sum()
zzl_num = df_final['RISK_LABEL'].apply(lambda x: 1 if '正常' not in x.split(',')[1] else 0).sum()

In [29]:
temp={}
temp['CHECK_TIME'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
temp['ORG_CODE'] = ORG_ODE
temp['MODEL_CODE'] = 'STOCK'
temp['CHILD_MODEL_CODE'] = 'ST3'
temp['ALARM_REASON'] = '发现{}起库存，{}起库存周转率事件'.format(ttl_num,zzl_num)
temp['ALARM_NUMBER'] = ttl_num+ zzl_num
pd.DataFrame([temp])

,CHECK_TIME,ORG_CODE,MODEL_CODE,CHILD_MODEL_CODE,ALARM_REASON,ALARM_NUMBER
0,2021-06-07 10:53:57,,STOCK,ST3,发现12起库存，5起库存周转率事件,17


In [11]:
RISK_ALARM=pd.DataFrame(columns=['ORG_CODE','MODEL_CODE','CHILD_MODEL_CODE','ALARM_REASON','ALARM_NUMBER','CHECK_TIME'])

In [24]:
RISK_ALARM[0] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [25]:
RISK_ALARM

,ORG_CODE,MODEL_CODE,CHILD_MODEL_CODE,ALARM_REASON,ALARM_NUMBER,CHECK_TIME,0


In [26]:
sql='''select * from BD_RISK_ALARM_ITEM'''
test = Read_Oracle().read_oracle(sql=sql,database='dbods')

2021-06-04 17:29:34.822 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.159s


In [27]:
test

,ID,ORG_CODE,MODEL_CODE,CHILD_MODEL_CODE,ALARM_REASON,ALARM_NUMBER,CHECK_TIME,ISCURRENT,LASTUPDATE
0,4,9527777,FINANCE,FN2,发现1起人工成本超支事件,1,2021-06-04 14:37:22,1,2021-06-04 14:37:23
1,2,91310000132612172J,STOCK,ST2,发现6起库存过高事件,6,2021-06-04 14:19:27,1,2021-06-04 14:19:39
2,3,91310000132612172J,STOCK,ST1,发现7起库存过高事件,7,2021-06-04 14:21:23,1,2021-06-04 14:22:07
3,0,91310000132612172J,STOCK,ST1,发现7起库存过高事件,7,2021-06-04 13:50:55,0,2021-06-04 14:22:07
4,1,91310000132612172J,STOCK,ST2,发现6起库存过高事件,6,2021-06-04 14:05:59,0,2021-06-04 14:19:40
5,7,91310000132612172J,PRODUCTION,PD2,发现1181起生产耗时过低事件,1181,2021-06-04 14:51:23,1,2021-06-04 14:53:52
6,8,91310000132612172J,PRODUCTION,PD2,发现77346起生产耗时过高事件,77346,2021-06-04 14:51:23,1,2021-06-04 14:53:52
7,9,91310115688774070B,WAREHOUSE,WH2,发现3起申报数量异常事件,3,2021-06-04 17:23:13,1,2021-06-04 17:23:20
8,10,91310115688774070B,WAREHOUSE,WH2,发现4起申报数量无法匹配，法定数量1异常事件,4,2021-06-04 17:23:13,1,2021-06-04 17:23:20
9,11,91310115688774070B,WAREHOUSE,WH2,发现97起贸易金额不符事件,97,2021-06-04 17:23:13,1,2021-06-04 17:23:20


In [2]:
# 整理预警明细数据，并写入数据库
RISK_ALARM = df_final[df_final['RISK_LABEL'] != '订单按时完成'].groupby(['RISK_LABEL'],as_index=False)['ID'].count()
RISK_ALARM = RISK_ALARM.rename(columns={'ID': 'ALARM_NUMBER'})
RISK_ALARM['ALARM_REASON'] = '发现' + RISK_ALARM['ALARM_NUMBER'].astype('str') + '起' + RISK_ALARM[
    'RISK_LABEL'] + '事件'
RISK_ALARM['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
RISK_ALARM['ORG_CODE'] = self.org_code
RISK_ALARM['MODEL_CODE'] = 'WAREHOUSE'
RISK_ALARM['CHILD_MODEL_CODE'] = 'WH2'
RISK_ALARM['ID'] = range(len(RISK_ALARM))
RISK_ALARM = RISK_ALARM[
    ['ID', 'ORG_CODE', 'MODEL_CODE', 'CHILD_MODEL_CODE', 'ALARM_REASON', 'ALARM_NUMBER', 'CHECK_TIME']]

if RISK_ALARM.empty:
    print('没有异常情况')
else:
    Write_Oracle().write_oracle('BD_RISK_ALARM_ITEM', RISK_ALARM, org_code=self.org_code,
                                alarm=['WAREHOUSE', 'WH2'])

NameError: name 'df_final' is not defined

In [20]:
Write_Oracle().write_oracle('BD_RISK_RESULT_STOCK_ST3',df_final,org_code=ORG_CODE)

2021-05-08 13:11:29.340 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.133s
2021-05-08 13:11:29.746 | INFO     | risk_models.config.write_config.write_func:write_oracle:128 - Processing... Writing 8 rows into database
2021-05-08 13:11:29.803 | INFO     | risk_models.config.write_config.write_func:write_oracle:135 - Insert data into BD_RISK_RESULT_STOCK_ST3 successfully! Total write time spent 0.604s
